In [1]:
import numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, accuracy_score,roc_curve,auc,precision_score,recall_score
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
import re

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
path_1="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG1.csv"
path_2="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG2.csv"
path_3="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG3.csv"
path_4="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG4.csv"
path_5="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG5.csv"
path_6="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG6.csv"
path_7="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG7.csv"
path_8="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG8.csv"
path_9="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG9.csv"
path_10="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG10.csv"
path_11="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG11.csv"
path_12="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG12.csv"
path_13="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG13.csv"
path_14="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG14.csv"
path_15="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG15.csv"
path_16="/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/TRAINING/re20000SDG16.csv"

df_1=pd.read_csv(path_1)
df_2=pd.read_csv(path_2)
df_3=pd.read_csv(path_3)
df_4=pd.read_csv(path_4)
df_5=pd.read_csv(path_5)
df_6=pd.read_csv(path_6)
df_7=pd.read_csv(path_7)
df_8=pd.read_csv(path_8)
df_9=pd.read_csv(path_9)
df_10=pd.read_csv(path_10)
df_11=pd.read_csv(path_11)
df_12=pd.read_csv(path_12)
df_13=pd.read_csv(path_13)
df_14=pd.read_csv(path_14)
df_15=pd.read_csv(path_15)
df_16=pd.read_csv(path_16)

df=pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15,df_16])

print(df.shape)
#dropping nulls
df.dropna(inplace=True)
print(df.shape)
#dropping duplicates
data = df.drop_duplicates(subset=["Text"], keep=False)
#data=df
print(data.shape)

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))
STOPWORDS.add("[TITLE]")
STOPWORDS.add("[ABSTRACT]")
print(len(STOPWORDS))

data = data.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
data['Text'] = data['Text'].apply(clean_text)
data['Text'] = data['Text'].str.replace('\d+', '')

X=data["Text"].values
Y=data["SDG"].values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 10)
print(X_train.shape,X_train.shape)
print(X_test.shape,Y_test.shape)

(305980, 2)
(305980, 2)
(179215, 2)
181


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: FutureWarning: The default value of regex will change from True to False in a future version.


(161293,) (161293,)
(17922,) (17922,)


In [31]:
data.to_csv("/content/drive/MyDrive/Colab Notebooks/training.csv")

In [16]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression(multi_class='ovr', solver='sag')),
              ])

#ovr = OneVsRestClassifier(nb)
nb.fit(X_train, Y_train)
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)
train_pred=nb.predict(X_train)
print('Train accuracy %s' % accuracy_score(train_pred, Y_train))
print('Test accuracy %s' % accuracy_score(y_pred, Y_test))
print(classification_report(Y_test, y_pred))

Train accuracy 0.8107915408604217
Test accuracy 0.7522597924338802
              precision    recall  f1-score   support

           1       0.72      0.67      0.70      1423
           2       0.76      0.79      0.77      1178
           3       0.79      0.89      0.84      1801
           4       0.66      0.69      0.68       463
           5       0.79      0.78      0.78      1215
           6       0.78      0.81      0.79      1086
           7       0.79      0.89      0.84      1526
           8       0.60      0.40      0.48       703
           9       0.75      0.73      0.74      1054
          10       0.70      0.65      0.67      1179
          11       0.72      0.77      0.74      1019
          12       0.70      0.58      0.63       732
          13       0.72      0.62      0.67       788
          14       0.83      0.83      0.83      1400
          15       0.77      0.79      0.78      1093
          16       0.72      0.76      0.74      1262

    accuracy 

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
xgb_Y_train = le.fit_transform(Y_train)
xgb_Y_test = le.fit_transform(Y_test)

In [12]:
import xgboost as xgboost

xgb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('xgboost', xgboost.XGBClassifier()),
              ])

xgb.fit(X_train, xgb_Y_train)
from sklearn.metrics import classification_report
y_pred = xgb.predict(X_test)
train_pred =xgb.predict(X_train)
print('Train accuracy %s' % accuracy_score(train_pred, xgb_Y_train))
print('Test accuracy %s' % accuracy_score(y_pred, Y_test))
print(classification_report(xgb_Y_test, y_pred))


Train accuracy 0.7605599747044199
Test accuracy 0.02259792433880147
              precision    recall  f1-score   support

           0       0.73      0.70      0.71      1423
           1       0.78      0.72      0.75      1178
           2       0.78      0.86      0.82      1801
           3       0.66      0.72      0.69       463
           4       0.74      0.82      0.78      1215
           5       0.76      0.85      0.80      1086
           6       0.79      0.86      0.83      1526
           7       0.65      0.47      0.54       703
           8       0.77      0.75      0.76      1054
           9       0.72      0.61      0.66      1179
          10       0.75      0.76      0.75      1019
          11       0.72      0.58      0.65       732
          12       0.68      0.74      0.71       788
          13       0.84      0.79      0.82      1400
          14       0.79      0.78      0.78      1093
          15       0.67      0.71      0.69      1262

    accuracy

In [35]:
test_data=pd.read_csv("/content/drive/Shareddrives/ResearchSDGs_CKidsDataFest2022/ML Training and Testing Datasets/XGBoost Model /testing.csv")

In [ ]:
test_data.head()

,Unnamed: 0,SDG,Text
0,0,1,[TITLE] The true costs of cesarean delivery fo...
1,1,1,[TITLE] The Association of Race Ethnicity and ...
2,2,1,[TITLE] Psychotic experiences among informal c...
3,3,1,[TITLE] The application of GIS in homelessness...
4,4,1,[TITLE] To Punish Parent or Palliate Governing...


In [24]:
# XGBoost Model on Test Data without removing stopwords
x_test=test_data["Text"].values
xgb_predictions = xgb.predict(x_test)+ 1
Y_true=test_data["SDG"].values
print('accuracy %s' % accuracy_score(xgb_predictions, Y_true))

accuracy 0.5094339622641509


In [36]:
# Logistic Regression Model on Test Data without removing stopwords
x_test=test_data["Text"].values
predictions=nb.predict(x_test)
Y_true=test_data["SDG"].values
print('accuracy %s' % accuracy_score(predictions, Y_true))

accuracy 0.6037735849056604


In [26]:
# remove stopwords in Test data
test_data['Text'] = test_data['Text'].apply(clean_text)
test_data['Text'] = test_data['Text'].str.replace('\d+', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [27]:
# XGBoost Model on Test Data after removing stopwords
x_test=test_data["Text"].values
xgb_predictions = xgb.predict(x_test)+ 1
Y_true=test_data["SDG"].values
print('accuracy %s' % accuracy_score(xgb_predictions, Y_true))

accuracy 0.5408805031446541


In [29]:
# Logistic Regression Model on Test Data after removing stopwords
predictions=nb.predict(x_test)
print('accuracy %s' % accuracy_score(predictions, Y_true))

accuracy 0.5849056603773585


In [ ]:
predictions

array([ 1,  1,  1,  1,  1,  1,  1, 16,  3,  1,  2, 10,  1,  1,  1,  2,  1,
        2,  2,  2,  2,  3,  3,  2,  5,  3,  3,  3, 10,  5, 16, 10, 16,  4,
       10,  1, 16,  4,  3, 10,  4,  5,  5, 10,  1,  5,  5,  2,  1,  1,  5,
        6,  6,  6,  6,  6,  6,  6,  3,  1, 16,  7, 13, 13,  7,  7,  7, 13,
       13,  9, 10,  1, 10,  8, 11,  1,  8,  9,  9, 11,  9,  7,  9,  9,  7,
        9,  9,  3, 10, 10,  1, 10,  5, 10, 10, 10, 16,  1,  1,  1, 11, 11,
       11, 11, 10,  1, 11, 11, 11,  6, 12, 12,  9, 15, 12,  7,  9, 12,  2,
       12, 13, 13,  2, 13, 13, 13, 13, 14, 14, 14, 14, 14, 13, 14, 14, 14,
       14, 15, 15, 15, 15, 15,  3, 13, 15, 15, 15, 14, 16, 10, 16, 16, 16,
        1,  5, 16, 16, 16, 14])

In [32]:
ML_predicted_sdgs=pd.DataFrame(xgb_predictions,columns=["XGBoost Predicted SDG"])

In [33]:
ML_predicted_sdgs.head()

,XGBoost Predicted SDG
0,1
1,1
2,1
3,1
4,1


In [34]:
ML_predicted_sdgs.to_csv("/content/drive/MyDrive/Colab Notebooks/xgb_predictions.csv")

In [37]:
LR_ML_predicted_sdgs=pd.DataFrame(predictions,columns=["LogReg Predicted SDG"])

In [38]:
LR_ML_predicted_sdgs.to_csv("/content/drive/MyDrive/Colab Notebooks/LogReg_predictions.csv")